In [21]:
!pip install mutagen tinytag

In [47]:
import pathlib
import os
import re

import mutagen as mg
from mutagen.id3 import ID3
import base64
from mutagen.oggvorbis import OggVorbis

from tinytag import TinyTag

import itertools
import shutil


In [23]:
cwd = pathlib.Path.cwd()

if cwd.name == 'ipod_recover_rename':
    print("Path is project root")
else:
    print('Current Working Directory is:', cwd)
    print("Recomend navigating to the project root in terminal and opening your chosen idea via the command 'code .'")
    raise NameError('Please correct current working directory to the project root')


Path is project root


In [24]:
old_path = cwd / "Music"
after_path = cwd / "After"

In [25]:
if not os.path.exists(after_path): 
    # if the After directory is not present  
    # then create it. 
    os.makedirs(after_path) 

In [26]:
f_list = os.listdir(old_path)
# copies music to after folder for processing of a copy of the music, ensuring no loss
for file in f_list:
    # prevents copying of the hidden files associated with ituns and ipods
    if file.startswith('._'):
        pass
    else:
        shutil.copytree(old_path / file, after_path / file)


In [27]:
# https://stackoverflow.com/questions/17547273/flatten-complex-directory-structure-in-python
#Move all files from music to (created) intermediary
def move_flatten(destination):
    all_files = []
    first_loop_pass = True
    for root, _dirs, files in os.walk(destination):
        if first_loop_pass:
            first_loop_pass = False
            continue
        for filename in files:
            all_files.append(os.path.join(root, filename))
    for filename in all_files:
        shutil.move(filename, destination)

In [28]:
move_flatten(after_path)

In [29]:
walk = list(os.walk(after_path))
for path, _, _ in walk[::]:
    if len(os.listdir(path)) == 0:
        # remove empty directories
        os.removedirs(path)


In [30]:
test_mp3 = after_path / 'CUYL.mp3'
test_m4a = after_path / 'XWHW.m4a'
test_m4p = after_path / 'DMZZ.m4p'
test_flac = after_path / 'ASCS.flac'
test_wav = after_path / 'GSDE.wav'
test_wma = after_path / 'AFES.wma'

test_files = [test_mp3, test_m4a, test_m4p, test_flac, test_wav, test_wma]

In [ ]:
# Handles names with / \ liek AC/DC 
def sanitize_string(file_dir_string):
    pattern = r'[/\\]'
    return re.sub(pattern, "_", file_dir_string)

In [54]:
def parse_metadata(f):
    file_path = after_path / f
    file = TinyTag.get(file_path)

    name_dict = {'album':'', 'artist':'', 'title':'', 'ext': ''}
    name_dict['artist'] = sanitize_string(file.artist)
    name_dict['album'] =  sanitize_string(file.album)
    name_dict['title'] = sanitize_string(file.title)
    name_dict['ext'] = file_path.suffix

    return name_dict



In [34]:
def move_and_rename(file, name_dict):
    # build folder and file paths
    artist_path = after_path / name_dict['artist']
    album_path = artist_path /  name_dict['album']
    title = name_dict['title']
    suffix = name_dict['ext']
    # check if album-artist folder paths exist, if not make them.
    # if artist folder DNE, make it
    if not os.path.isdir(artist_path):
        os.mkdir(artist_path)
    # if album folder DNE, make it
    if not os.path.isdir(album_path):
        os.mkdir(album_path)
    # move the song to the album-artist folder and rename it
    renamed_ordered_path = album_path / f'{title}{suffix}'
    shutil.move(after_path / file, renamed_ordered_path)

    return renamed_ordered_path

In [55]:
files = os.listdir(after_path)
count = 1
for file in files:
    print(f'working on {count}/{len(files)}: {file}')
    name_dict = parse_metadata(file)
    new = move_and_rename(file, name_dict)
    print('moved and renamed to:', new)
    count += 1
    # TODO: Handle missing metadata

working on 1/580: ARMA.m4a
moved and renamed to: /home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/After/AC_DC/AC_DC Live: Collector's Edition [Disc 1]/Fire Your Guns [Live].m4a
working on 2/580: EUCU.m4a
moved and renamed to: /home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/After/Al Green/Al Green Explores Your Mind/The City.m4a
working on 3/580: WUWG.mp3
moved and renamed to: /home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/After/ANDRE NICKATINA & MAC DRE /TALE OF 2 ANDRES/OUTTA CONTROL.mp3
working on 4/580: CWQG.m4a
moved and renamed to: /home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/After/AC*DC/Powerage/Sin City.m4a
working on 5/580: FSUV.m4a
moved and renamed to: /home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/After/Behemoth/Demigod+Satanica/Track 07.m4a
working on 6/580: FMGI.m4a
moved and renamed to: /home/mox/Documents/coding_projects/personal_projects/

TypeError: expected string or bytes-like object, got 'NoneType'

TODO: 

1. rename files to artist_album_title.ext
2. Add files to artist-album-song structure
    1. walk file list and make [artist, album, title.ext]
    2. check if artist-album folders exist, make them if DNE 
    3. rename to title.ext and move to appropriate folder tree

In [ ]:
folder = r'AC/DC'
path = after_path / folder
print(os.path.isdir(path))
os.mkdir(path)

False


FileNotFoundError: [Errno 2] No such file or directory: '/home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/After/AC/DC'